In [54]:
import pandas as pd #자료의 데이터프레임화
import random #타임슬립에 랜덤숫자적용
import time #타입슬립
import json #파일저장
import os
#인스타그램 접속
from random_user_agent.params import SoftwareName, OperatingSystem 
from random_user_agent.user_agent import UserAgent
#웹드라이버 및 화면조종
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.by import By #로딩함수에 적용
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys

# from eunjeon import Mecab
import unicodedata #이모티콘
# import pyodbc
import re #문자분류
import collections

class IG_Bot:
    '''로봇처럼 보이기 방지'''
    def __init__(self, username, password):
        software_names= [SoftwareName.CHROME.value]
        operating_systems=[OperatingSystem.WINDOWS.value]
        
        #Randomized User ID
        user_agent_rotator= UserAgent(software_name = software_names,
                                     operating_systems=operating_systems,
                                     limit=100)
        user_agent= user_agent_rotator.get_random_user_agent()
        chrome_options=Options()
#         chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sendbox")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument(f'user-agent={user_agent}')
        path='/Users/jaminkim/python/chromedriver'
        
        cookies_file_path='/Users/jaminkim/python/말뭉치/cookies.json'
        cookie_websites=['https://www.tistory.com/']
        self.cookies_file_path= cookies_file_path
        self.cookie_websites= cookie_websites
        self.driver=webdriver.Chrome(chrome_options=chrome_options,executable_path=path)
        
        self.driver.get('https://www.tistory.com/')
        self.driver.maximize_window() # 전체화면
        
        time.sleep(1.5)

        try:
            cookies=json.load(open(self.cookies_file_path,'rb'))
            for website in self.cookie_websites:
                for cookie in cookies:
                    self.driver.add_cookie(cookie)
                self.driver.refresh()
        except Exception as e:
            print(str(e))
       
    def loading(self,xpath):
        '''페이지로딩 기다리기'''
        WebDriverWait(self.driver,60).until(EC.presence_of_element_located((By.XPATH,xpath)))
    
    def save_cookies(self):
        '''쿠키저장'''
        json.dump(self.driver.get_cookies(),open('cookies.json','w'))
    
    def load_cookies(self):
        '''쿠키로드'''
        cookies= json.load(open('cookies.json','rb'))
        for cookie in cookies:
            self.driver.add_cookie(cookie)
            
    def login(self):
        try:
            self.driver.find_element_by_xpath('//*[@id="kakaoHead"]/div/div[2]/div/a').click() # 시작하기
            self.driver.find_element_by_xpath('//*[@id="cMain"]/div/div/div/a[1]/span[2]').click() # 카카오로 로그인 하기
            self.driver.find_element_by_xpath('//*[@id="id_email_2"]').send_keys(username) # ID
            self.driver.find_element_by_xpath('//*[@id="id_password_3"]').send_keys(password) #PW
            self.driver.find_element_by_xpath('//*[@id="login-form"]/fieldset/div[8]/button[1]').click() # 로그인
        except:
            pass
        
class Crawler(IG_Bot):
    global insta_dict
    global number
    global categorys
    number=957
    insta_dict=[]
    categorys = {'라이프':'life', '여행.맛집':'travel', '문화.연예':'culture', 'IT':'it', '스포츠':'sports', '시사':'current'}
    
    def run(self):
        count=1

        while True:
            try:
                category=int(input('1. 라이프 2. 여행.맛집 3. 문화.연예 4. IT 5. 스포츠 6. 시사\n카테고리 번호를 입력하시오 > '))
            except ValueError:
                print('숫자를 입력해주세요.')
                continue
            else:
                if category <= 0 or category >= 7:
                    print('1-6의 숫자를 입력해주세요.')
                    continue
                else:
                    break

        s_time=time.time()
        
        category_values = list(categorys.values())
        self.driver.get('https://www.tistory.com/category/'+category_values[category-1]) # 카테고리 페이지로 이동
        
        while count <= 10:
            try:
                self.driver.find_element_by_xpath(f'//*[@id="mArticle"]/div[3]/ul/li[{count}]/a').click() # 게시물 클릭
            except NoSuchElementException:
                body = self.driver.find_element_by_xpath("/html/body")
                body.send_keys(Keys.PAGE_DOWN) # 페이지 다운
                continue
            else:
                self.driver.switch_to_window(self.driver.window_handles[1]) 
                self.driver.get_window_position(self.driver.window_handles[1]) # 새창 넘어가기

                self.crawl()

                self.driver.close() # 새창 닫기
                self.driver.switch_to.window(self.driver.window_handles[0]) # 기존 창으로 돌아오기
                count += 1

                time.sleep(random.randrange(0,5))

#         print('id 저장 개수 :{}개'.format(len(insta_dict['id'])))
#         print('텍스트 저장 개수 :{}개'.format(len(insta_dict['text'])))

        e_time=time.time()
        (f'{e_time-s_time}초 걸렸습니다.')
        
        #insta_dict로 저장한 데이터 JSON파일로 저장
        with open(f'tstory_{category_values[category-1]}_data.json', 'w') as f:
            json.dump(insta_dict, f, indent=4) 
        
    
    def crawl(self):
        extract={"id": "IFRW21", 
         "metadata": { "title": "휘뚜마뚜 웹문서 말뭉치 IFRW21", 
                      "creator": "김자민", 
                      "distributor": "휘뚜마뚜", 
                      "year": "2021", 
                      "category": "티스토리 > 음식", 
                      "annotation_level": [ "원시" ], 
                      "sampling": "텍스트 추출" }, 
         "document": {"metadata": { "title": "무제", 
                                     "author": "", 
                                     "date": "20211110" }, 
                        "content": {"text": ""}}
                    }
        global number
#         ## id 정보 수집
#         info_id = self.driver.find_element_by_xpath('//*[@id="header"]/div/h1/a').text
#         extract['document']['metadata']['author']=info_id
        
        ##text 정보수집
        try:
            raw_info = self.driver.find_element_by_xpath('//*[@id="container"]').text.split()
            raw_info = raw_info[1:len(raw_info)-1]
            text=[]
            
            for i in raw_info:
                if '#' in i:
                    pass
                else:
                    text.append(i)
            clean_text = ' '.join(text)
            extract['document']['content']['text']=clean_text
        except:
            text=''
            extract['document']['content']['text']=text
        
    #일련번호
        s_number=str(number).zfill(8)
        extract['id']=extract['id'][:6]+s_number
        number+=1
        
        #insta_dict
        insta_dict.append(extract)
        
        '''c드라이브에 IG폴더 만들고 secret.txt 파일에 윗줄에 아이디 2번째줄에 비밀번호 입력'''    
if __name__=='__main__':
    "Main (runs through classes)"
    data=pd.read_csv('/Users/jaminkim/python/말뭉치/secret.txt',header=None)
    username=data.iloc[0][0]
    password=data.iloc[1][0]
    
    '''처음실행하고 쿠키저장'''
def first_login():
    if not os.path.exists("cookies.json"):
        bot=IG_Bot(username,password)
        bot.login()
        bot.save_cookies()
    else:
        pass

    '''쿠키저장 후 크롤러 가동하기'''
def run_():
    # '''Crawler'''
    bot=Crawler(username,password)
    bot.run()
    
first_login()
run_()

<ipython-input-54-ca7e17a298c1>:48: DeprecationWarning: use options instead of chrome_options
  self.driver=webdriver.Chrome(chrome_options=chrome_options,executable_path=path)


1. 라이프 2. 여행.맛집 3. 문화.연예 4. IT 5. 스포츠 6. 시사
카테고리 번호를 입력하시오 > 2


<ipython-input-54-ca7e17a298c1>:125: DeprecationWarning: use driver.switch_to.window instead
  self.driver.switch_to_window(self.driver.window_handles[1])
